[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/reznikmm/ada-howto/advent-2020?filepath=%2Fhome%2Fjovyan%2Fnb%2F04%2F04.ipynb)

# Day 4

Firstly, we need some standard packages to read the text file and process strings.

In [1]:
with Ada.Text_IO;
with Ada.Strings.Fixed;
with Ada.Strings.Maps;

Let's define an enumeration type for pasport fields. We keep validation flag for each field.

In [2]:
type Field_Kind is
  (byr,  --  (Birth Year)
   iyr,  --  (Issue Year)
   eyr,  --  (Expiration Year)
   hgt,  --  (Height)
   hcl,  --  (Hair Color)
   ecl,  --  (Eye Color)
   pid,  --  (Passport ID)
   cid);   --  (Country ID)

type Field_Set is array (Field_Kind) of Boolean;

A pasport is valid if required fields are present and valid:

In [3]:
function Is_Valid (Pasport : Field_Set) return Boolean is
begin
   return Pasport (byr .. pid) = (byr .. pid => True);
end Is_Valid;

To validate fields we use some character sets:

In [4]:
Digit  : Ada.Strings.Maps.Character_Set :=
   Ada.Strings.Maps.To_Set ("0123456789");
Hex    : Ada.Strings.Maps.Character_Set :=
  Ada.Strings.Maps.To_Set ("0123456789abcdef");
Space  : constant Ada.Strings.Maps.Character_Set :=
     Ada.Strings.Maps.To_Set (" ");

Now we can define rather long, but straightforward function

In [5]:
function Is_Valid_Field (Text : String; Kind : Field_Kind) return Boolean is
      
   function Is_Number (Text : String) return Boolean is
   begin
      return Ada.Strings.Fixed.Count (Text, Digit) = Text'Length;
   end Is_Number;
      
begin
   case Kind is
      when byr =>
         return Is_Number (Text)
           and then Text'Length = 4
           and then Integer'Value (Text) in 1920 .. 2002;
      when iyr =>
         return Is_Number (Text)
           and then Text'Length = 4
           and then Integer'Value (Text) in 2010 .. 2020;
      when eyr =>
         return Is_Number (Text)
           and then Text'Length = 4
           and then Integer'Value (Text) in 2020 .. 2030;
      when hgt =>
         if Text'Length > 3
           and then Is_Number (Text (Text'First .. Text'Last - 2))
           and then Text (Text'Last - 1 .. Text'Last) in "cm" | "in"
         then
            declare
               Value : Natural :=
                 Integer'Value (Text (Text'First .. Text'Last - 2));
            begin
               if Text (Text'Last) = 'm' then
                  return Value in 150 .. 193;
               else
                  return Value in 59 .. 76;
               end if;
            end;
         end if;
      when hcl =>
         return Text'Length = 7
           and then Text (Text'First) = '#'
           and then Ada.Strings.Fixed.Count (Text, Hex) = 6;
      when ecl =>
         return Ada.Strings.Fixed.Index
           ("amb blu brn gry grn hzl oth", Text) > 0;
      when pid =>
         return Text'Length = 9 and then Is_Number (Text);
      when cid => 
         return True;
   end case;
   
   return False;
end Is_Valid_Field;

Now declare a file to read input:

In [6]:
Input : Ada.Text_IO.File_Type;

Having a field as a text we can decode it this way:

In [7]:
procedure Read_Field (Text : String; Result : in out Field_Set) is
   Kind : Field_Kind;
begin
   pragma Assert (Text (Text'First + 3) = ':');
   Kind := Field_Kind'Value (Text (Text'First .. Text'First + 2));
   Result (Kind) :=
     Is_Valid_Field (Text (Text'First + 4 .. Text'Last), Kind);
end Read_Field;

Now we can read pasport data as sequence of non-empty lines. To split fields we can use `Find_Token` procedure providing `Space` as separator character set.

In [8]:
procedure Read_Pasport (Result : out Field_Set) is
begin
   Result := (others => False);

   while not Ada.Text_IO.End_Of_File (Input) loop
      declare
         Line : String := Ada.Text_IO.Get_Line (Input);
         First : Positive;
         Last  : Natural := 0;
      begin
         exit when Line = "";

         while Last < Line'Last loop
            Ada.Strings.Fixed.Find_Token
              (Source => Line,
               Set    => Space,
               From   => Last + 1,
               Test   => Ada.Strings.Outside,
               First  => First,
               Last   => Last);

            exit when Last < First;

            Read_Field (Line (First .. Last), Result);
         end loop;
      end;
   end loop;
end Read_Pasport;

Declare `Result` to keep valid pasport count.

In [9]:
Result : Natural := 0;

Now we just loop over input lines reading pasports and validating them

In [10]:
Ada.Text_IO.Open (Input, Ada.Text_IO.In_File, "/home/jovyan/md/04/input");

while not Ada.Text_IO.End_Of_File (Input) loop
   declare
      Pasport : Field_Set;
   begin
      Read_Pasport (Pasport);

      if Is_Valid (Pasport) then
         Result := Result + 1;
      end if;
   end;
end loop;
   
Ada.Text_IO.Put_Line (Result'Image);

 172


----
[Back to Table of Contents](https://github.com/reznikmm/ada-howto/tree/advent-2020)